<a href="https://colab.research.google.com/github/acastellanos-ie/natural_language_processing/blob/master/language%20modelling%20elmo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab Configuration

**Execute this steps to configure the Google Colab environment in order to execute this notebook. It is not required if you are executing it locally and you have properly configured your local environment according to what explained in the Github Repository.**

The first step is to clone the repository to have access to all the data and files

In [ ]:
! git clone https://github.com/acastellanos-ie/MBD-EN-BL-ENE-2020-J-1.git

fatal: destination path 'MBD-EN-BL-ENE-2020-J-1' already exists and is not an empty directory.


Install the requirements

In [ ]:
! pip install -Uqqr MBD-EN-BL-ENE-2020-J-1/requirements.txt

     |████████████████████████████████| 1.5MB 4.0MB/s 
     |████████████████████████████████| 10.4MB 22.5MB/s 
     |████████████████████████████████| 12.0MB 336kB/s 
     |████████████████████████████████| 9.9MB 44.6MB/s 
     |████████████████████████████████| 358kB 30.5MB/s 
     |████████████████████████████████| 204kB 42.5MB/s 
     |████████████████████████████████| 727kB 35.2MB/s 
     |████████████████████████████████| 454.3MB 35kB/s 
     |████████████████████████████████| 25.3MB 1.5MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 2.3MB 34.9MB/s 
     |████████████████████████████████| 1.8MB 30.9MB/s 
     |████████████████████████████████| 1.1MB 40.7MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 1.2MB 33.9MB/s 
     |████████████████████████████████| 983kB 21.7MB/s 
     |████████████████████████████████| 266kB 38.8MB/s

Ensure that you have the GPU runtime activated:

![](https://miro.medium.com/max/3006/1*vOkqNhJNl1204kOhqq59zA.png)

Now you have everything you need to execute the code in Colab

# Language Modelling

Import the libraries we’ll be using throughout our notebook:


In [ ]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

**Import the corpus**

Here’s a breakdown of the dataset we have:

The train set contains 7,920 tweets
The test set contains 1,953 tweets

You can download the dataset from this [page](https://datahack.analyticsvidhya.com/contest/linguipedia-codefest-natural-language-processing-1/#data_dictionary). Note that you will have to register or sign-in to do so.

Caution: Most profane and vulgar terms in the tweets have been replaced with “$&@*#”. However, please note that the dataset might still contain text that could be considered profane, vulgar, or offensive.


In [ ]:
train = pd.read_csv("MBD-EN-BL-ENE-2020-J-1/language_modelling/dataset/train_2kmZucJ.csv")
test = pd.read_csv("MBD-EN-BL-ENE-2020-J-1/language_modelling/dataset/test_oJQbWVk.csv")

train.shape, test.shape

((7920, 3), (1953, 2))

The train set has 7,920 tweets while the test set has only 1,953. Now let’s check the class distribution in the train set:

In [ ]:
train['label'].value_counts(normalize = True)

0    0.744192
1    0.255808
Name: label, dtype: float64

Here, 1 represents a negative tweet while 0 represents a non-negative tweet.

Let’s take a quick look at the first 5 rows in our train set:

In [ ]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


## Text Cleaning and Preprocessing

We would have a clean and structured dataset to work with in an ideal world. But things are not that simple in NLP (yet).

We need to spend a significant amount of time cleaning the data to make it ready for the model building stage. Feature extraction from the text becomes easy and even the features contain more information. You’ll see a meaningful improvement in your model’s performance the better your data quality becomes.

So let’s clean the text we’ve been given and explore it.

In [ ]:

# remove URL's from train and test
train['clean_tweet'] = train['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

test['clean_tweet'] = test['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

train['clean_tweet'] = train['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

# convert text to lowercase
train['clean_tweet'] = train['clean_tweet'].str.lower()
test['clean_tweet'] = test['clean_tweet'].str.lower()

# remove numbers
train['clean_tweet'] = train['clean_tweet'].str.replace("[0-9]", " ")
test['clean_tweet'] = test['clean_tweet'].str.replace("[0-9]", " ")

# remove whitespaces
train['clean_tweet'] = train['clean_tweet'].apply(lambda x:' '.join(x.split()))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ' '.join(x.split()))

Let’s have a quick look at the original tweets vs our cleaned ones:

In [ ]:
train.sample(10)

,id,label,tweet,clean_tweet
2006,2007,0,It's a great day for HOCKEY #LETS #GO @penguins #bruins !!! #sky #apple #iphone4s #igers… http://instagram.com/p/aMOdwzjD8N/,it's a great day for hockey lets go penguins bruins sky apple iphone s igers…
5621,5622,0,Hey Guys! Look Apple-iPhone-X-256-GB-Space-Gray-Unlocked http://zpr.io/6EdnM #money #today #life #amazon #birthday #sale #android #windows #nokia #samsung #iphone #phone #smartphone #motorola #off...,"hey guys look appleiphonex gbspacegrayunlocked money today life amazon birthday sale android windows nokia samsung iphone phone smartphone motorola offers bestprice aug , am"
1678,1679,0,#redhair #red #loca #me #kim #addicted #hair#photoaday #iphone #igaddict #instahub… http://instagram.com/p/aJuTwiLNPr/,redhair red loca me kim addicted hairphotoaday iphone igaddict instahub…
2716,2717,1,Fuck you #apple you bring that stupid #ipadmini and we still have the shitty maps...,fuck you apple you bring that stupid ipadmini and we still have the shitty maps...
7567,7568,1,I'm not even sure why I own #Apple products BECAUSE I REALLY HATE YOUR WHOLE COMPANY AND YOUR CRAPPY COMPUTERS.,i'm not even sure why i own apple products because i really hate your whole company and your crappy computers.
6971,6972,0,New ReallyReal v2.1 #Photobooth for #iphone #ipad #Dating - http://reallyreal.com/ #college #sorority pi pic.twitter.com/6oWUvaIqCY,new reallyreal v . photobooth for iphone ipad dating college sorority pi pic.twitter.com owuvaiqcy
691,692,1,Stupid iPhone.. Fragile $&@*# whore.. I hate you apple.. Why can't the back be titanium again or what ever the fuck it was.. iphone,stupid iphone.. fragile whore.. i hate you apple.. why can't the back be titanium again or what ever the fuck it was.. iphone
5157,5158,0,#petstagram #iphone #instagram #cute #puppy #sleepy #adorable #dog #silly #Team-Follow-me http://instagr.am/p/Tl8ex/,petstagram iphone instagram cute puppy sleepy adorable dog silly teamfollowme
1077,1078,1,Now I have to change my iCloud email log in. Surely they are all connected?!,now i have to change my icloud email log in. surely they are all connected
1734,1735,0,Top5 popular tags: 1^ #AfterIPLisNationalized 2^ #IPL 3^ #iphone 4^ #cricket 5^ http://india.twirus.com,top popular tags afteriplisnationalized ipl iphone cricket


Check out the above columns closely. The tweets in the ‘clean_tweet’ column appear to be much more legible than the original tweets.

However, I feel there is still plenty of scope for cleaning the text. I encourage you to explore the data as much as you can and find more insights or irregularities in the text.


## Preparing ELMo Vectors
We will now import the pretrained ELMo model. A note of caution – the model is over 350 mb in size so it might take you a while to download this

In [ ]:
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

I will first show you how we can get ELMo vectors for a sentence. All you have to do is pass a list of string(s) in the object elmo.

In [ ]:
# just a random sentence
x = ["Roasted ants are a popular snack in Columbia"]

# Extract ELMo features 
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

embeddings.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


TensorShape([1, 8, 1024])


The output is a 3 dimensional tensor of shape (1, 8, 1024):

The first dimension of this tensor represents the number of training samples. This is 1 in our case.

The second dimension represents the maximum length of the longest string in the input list of strings. Since we have only 1 string in our input list, the size of the 2nd dimension is equal to the length of the string – 8

The third dimension is equal to the length of the ELMo vector
Hence, every word in the input sentence has an ELMo vector of size 1024.

Let’s go ahead and extract ELMo vectors for the cleaned tweets in the train and test datasets. However, to arrive at the vector representation of an entire tweet, we will take the mean of the ELMo vectors of constituent terms or tokens of the tweet.

Let’s define a function for doing this:

In [ ]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

You might run out of computational resources (memory) if you use the above function to extract embeddings for the tweets in one go. 

As a workaround, split both train and test set into batches of 100 samples each. Then, pass these batches sequentially to the function elmo_vectors().

I will keep these batches in a list:

In [ ]:
list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

Now, we will iterate through these batches and extract the ELMo vectors. Let me warn you, this will take a long time.

In [ ]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['clean_tweet']) for x in list_train]
elmo_test = [elmo_vectors(x['clean_tweet']) for x in list_test]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Once we have all the vectors, we can concatenate them back to a single array:



In [ ]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

I would advice you to save these arrays as it took us a long time to get the ELMo vectors for them. We will save them as pickle files:

In [ ]:
# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

Use the following code to load them back:



In [ ]:
# load elmo_train_new
pickle_in = open("elmo_train_03032019.pickle", "rb")
elmo_train_new = pickle.load(pickle_in)

# load elmo_train_new
pickle_in = open("elmo_test_03032019.pickle", "rb")
elmo_test_new = pickle.load(pickle_in)

## Model Building and Evaluation
Let’s build our NLP model with ELMo!

We will use the ELMo vectors of the train dataset to build a classification model. Then, we will use the model to make predictions on the test set. But before all of that, split `elmo_train_new` into training and validation set to evaluate our model prior to the testing phase.

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  train['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)

Since our objective is to set a baseline score, we will build a simple logistic regression model using ELMo vectors as features:

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression(max_iter=1000)
lreg.fit(xtrain, ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Prediction time! First, on the validation set:



In [ ]:
preds_valid = lreg.predict(xvalid)


We will evaluate our model by the F1 score metric since this is the official evaluation metric of the contest.

In [ ]:
f1_score(yvalid, preds_valid)


0.7821901323706377

Output: 0.789976

The F1 score on the validation set is pretty impressive. Now let’s proceed and make predictions on the test set:

In [ ]:
# make predictions on test set
preds_test = lreg.predict(elmo_test_new)

In [ ]:
# prepare submission dataframe
sub = pd.DataFrame({'id':test['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv("sub_lreg.csv", index=False)

These predictions give us a score of 0.875672 on the public leaderboard. That is frankly pretty impressive given that we only did fairly basic text preprocessing and used a very simple model. Imagine what the score could be with more advanced techniques. Try them out on your end and let me know the results!